In [1]:
import numpy as np
import pandas as pd
import os

# Plotting Libs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import collections
import nltk
import time
nltk.download('punkt')

# magic function
%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jaanavnalina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#To calculate duration of process
sTime = time.time()

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
#df_tags = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Projects/01 Multiclass Multilabel prediction For stack overflow Questions/Tags.csv',nrows=75019, encoding='iso-8859-1') #Considered 20% of data for building model
#df_tags = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Projects/01 Multiclass Multilabel prediction For stack overflow Questions/Tags.csv',encoding='iso-8859-1') #Considered 20% of data for building model
df_tags = pd.read_csv('Tags.csv',encoding='iso-8859-1') #Considered 20% of data for building model

In [5]:
df_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750994 entries, 0 to 3750993
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Tag     object
dtypes: int64(1), object(1)
memory usage: 57.2+ MB


In [6]:
topTen = list(df_tags['Tag'].value_counts().head(10).index)
topTen

['javascript',
 'java',
 'c#',
 'php',
 'android',
 'jquery',
 'python',
 'html',
 'c++',
 'ios']

In [7]:
tag_topTen = df_tags[df_tags['Tag'].isin(topTen)]
tag_topTen.info()

<class 'pandas.core.frame.DataFrame'>
Index: 826739 entries, 14 to 3750989
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Id      826739 non-null  int64 
 1   Tag     826739 non-null  object
dtypes: int64(1), object(1)
memory usage: 18.9+ MB


In [8]:
tag_topTen

,Id,Tag
14,260,c#
18,330,c++
28,650,c#
35,930,c#
39,1010,c#
...,...,...
3750974,40143150,javascript
3750981,40143190,python
3750984,40143210,php
3750987,40143340,android


In [9]:
def add_tags(question_id):
    return tag_topTen[tag_topTen['Id'] == question_id['Id']].Tag.values

In [10]:
topTen = tag_topTen.apply(add_tags, axis=1)

In [11]:
topTen.info()

<class 'pandas.core.series.Series'>
Index: 826739 entries, 14 to 3750989
Series name: None
Non-Null Count   Dtype 
--------------   ----- 
826739 non-null  object
dtypes: object(1)
memory usage: 12.6+ MB


In [12]:
tag_topTen.head()

,Id,Tag
14,260,c#
18,330,c++
28,650,c#
35,930,c#
39,1010,c#


In [13]:
tag_topTen=pd.concat([tag_topTen, topTen.rename('Tags')], axis=1)
tag_topTen.head()

,Id,Tag,Tags
14,260,c#,[c#]
18,330,c++,[c++]
28,650,c#,[c#]
35,930,c#,[c#]
39,1010,c#,[c#]


In [14]:
tag_topTen.drop(["Tag"], axis=1, inplace=True)
tag_topTen.shape

(826739, 2)

In [15]:
tag_topTen.head()

,Id,Tags
14,260,[c#]
18,330,[c++]
28,650,[c#]
35,930,[c#]
39,1010,[c#]


In [16]:
tag_topTen=tag_topTen.loc[tag_topTen.astype(str).drop_duplicates().index]

In [17]:
tag_topTen.head()

,Id,Tags
14,260,[c#]
18,330,[c++]
28,650,[c#]
35,930,[c#]
39,1010,[c#]


In [18]:
#Cleaning questions
#question = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Projects/01 Multiclass Multilabel prediction For stack overflow Questions/Questions.csv',encoding='iso-8859-1',nrows=25285) #20% of data for testing purpose
#question = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Projects/01 Multiclass Multilabel prediction For stack overflow Questions/Questions.csv',encoding='iso-8859-1') #20% of data for testing purpose
question = pd.read_csv('Questions.csv',encoding='iso-8859-1') #20% of data for testing purpose

In [19]:
question.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [20]:
question = question[['Id','Title','Body']]

In [21]:
question.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264216 entries, 0 to 1264215
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   Id      1264216 non-null  int64 
 1   Title   1264216 non-null  object
 2   Body    1264216 non-null  object
dtypes: int64(1), object(2)
memory usage: 28.9+ MB


In [22]:
question.head(10)

,Id,Title,Body
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...
5,330,Should I use nested classes in this case?,<p>I am working on a collection of classes use...
6,470,Homegrown consumption of web services,<p>I've been writing a few web services for a ...
7,580,Deploying SQL Server Databases from Test to Live,<p>I wonder how you guys manage deployment of ...
8,650,Automatically update version number,<p>I would like the version property of my app...
9,810,Visual Studio Setup Project - Per User Registr...,<p>I'm trying to maintain a Setup Project in <...


In [23]:
import re
def remHTMLElements(body):
    regex = re.compile(r'<.*?>')
    return re.sub(regex, '', body)

In [24]:
question['Body'] = question['Body'].apply(remHTMLElements)

In [25]:
question.sample(100)

,Id,Title,Body
211189,7992420,Is there any way to get the size of a c function?,I want to know if there is a way to get the si...
198403,7569100,Resize Image PNG With transparence,I want to resize an image PNG with transparenc...
302837,10997980,Winform class to main program,How would I get the main program to recognize ...
650793,22313840,jQuery Rotator making page jump when the first...,Hello I am working on this page http://rcmhost...
486632,16986780,List untested methods with Visual Studio or Re...,Let's just say that once upon a time you had a...
...,...,...,...
630008,21659040,PHP datetime() returning one hour less time af...,I just found out that if I call datetime() reg...
973736,32316190,Xamarin.IOS Toggle Button?,Is there toggle button for xamarin.ios? If not...
923999,30812600,Android JUnit: assert inside the handler,I am trying make a assert inside of handler bu...
1213120,38847930,Objective C to Swift - DoWithBlock,I am trying to only create an array in Swift w...


In [26]:
# question.to_csv('/content/drive/My Drive/Colab Notebooks/Projects/01 Multiclass Multilabel prediction For stack overflow Questions/question_clean.csv',index=False)
question.to_csv('question_clean.csv',index=False)

In [27]:
# question = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Projects/01 Multiclass Multilabel prediction For stack overflow Questions/question_clean.csv', encoding='iso-8859-1')
question = pd.read_csv('question_clean.csv', encoding='iso-8859-1')
question.head()

,Id,Title,Body
0,80,SQLStatement.execute() - multiple queries in o...,I've written a database generation script in S...
1,90,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...
2,120,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...
3,180,Function for creating color wheels,This is something I've pseudo-solved many time...
4,260,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...


In [28]:
cleanedData=pd.merge(question, tag_topTen, on='Id')
print(cleanedData.shape)

(706336, 4)


In [29]:
cleanedData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706336 entries, 0 to 706335
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Id      706336 non-null  int64 
 1   Title   706336 non-null  object
 2   Body    706336 non-null  object
 3   Tags    706336 non-null  object
dtypes: int64(1), object(3)
memory usage: 21.6+ MB


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization, GRU,concatenate
from keras.models import Model

/Users/jaanavnalina/Learning/IITK/DeepLearning/01_StackOverFlow/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [31]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(cleanedData['Tags'])
labels = multilabel_binarizer.classes_
labels

array(['android', 'c#', 'c++', 'html', 'ios', 'java', 'javascript',
       'jquery', 'php', 'python'], dtype=object)

In [32]:
cleanedData['Tags']

0                 [c#]
1                [c++]
2                 [c#]
3                 [c#]
4                 [c#]
              ...     
706331    [javascript]
706332        [python]
706333           [php]
706334       [android]
706335    [javascript]
Name: Tags, Length: 706336, dtype: object

In [33]:
train,test=train_test_split(cleanedData,test_size=0.25,random_state=24)
#train,test=train_test_split(total[:550000],test_size=0.25,random_state=24)

In [34]:
train.shape,test.shape

((529752, 4), (176584, 4))

In [35]:
train.head()

,Id,Title,Body,Tags
673359,38585100,How to get mshtml.IHTMLDocument6 or mshtml.IHT...,I am using IE11 in Windows 7.\nThen I added a ...,[c#]
214427,14394610,Do I have to typedef on every Header file?,Let's say I have an std::vector of std::string...,[c++]
512204,30670650,How to apply an expression tree for IQueryable?,Is there any way to apply a manually created e...,[c#]
20451,2269980,Why can't I call this function in Java?,import static com.example.hello.Tools.*;\npubl...,[java]
475583,28734010,pattern to dictionary of lists Python,I have a file like this\n\nmodule1 instance1(....,[python]


In [36]:
X_train_Title=train['Title']
X_train_Body=train['Body']
y_train=multilabel_binarizer.transform(train['Tags'])
X_test_Title=test['Title']
X_test_Body=test['Body']
y_test=multilabel_binarizer.transform(test['Tags'])

In [37]:
X_train_Title.head()

673359    How to get mshtml.IHTMLDocument6 or mshtml.IHT...
214427           Do I have to typedef on every Header file?
512204      How to apply an expression tree for IQueryable?
20451               Why can't I call this function in Java?
475583                pattern to dictionary of lists Python
Name: Title, dtype: object

In [38]:
y_train

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [39]:
train['Title']

673359    How to get mshtml.IHTMLDocument6 or mshtml.IHT...
214427           Do I have to typedef on every Header file?
512204      How to apply an expression tree for IQueryable?
20451               Why can't I call this function in Java?
475583                pattern to dictionary of lists Python
                                ...                        
478609    Entity Framework loses track of navigation pro...
516439    Dependency Injection into Spring non-managed b...
211136              execute script based on variable states
899                      Multiple Data Tables in PHP/MySQL?
242082                  How can I get JLabel's border color
Name: Title, Length: 529752, dtype: object

In [40]:
len(word_tokenize("Multiple Data Tables in PHP/MySQL?"))

6

In [41]:
sentLensTitle=[]
for line in train['Title']:
    sentLensTitle.append(len(word_tokenize(line)))
max(sentLensTitle)

60

In [42]:
maxLenTitleTemp = int(np.quantile(sentLensTitle,0.97))

In [43]:
maxLenTitleTemp

18

In [44]:
maxLenTitle = maxLenTitleTemp + (10-(maxLenTitleTemp % 10))

In [45]:
maxLenTitle

20

In [46]:
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train_Title)
sequences_train_Title = tok.texts_to_sequences(X_train_Title)

In [47]:
sequences_train_Title[:10]

[[4, 1, 24, 13295, 35155, 39, 13295, 35156],
 [35, 15, 165, 1, 3350, 10, 435, 326, 23],
 [4, 1, 690, 19, 332, 620, 14, 3998],
 [46, 134, 15, 98, 67, 31, 2, 21],
 [438, 1, 424, 7, 579, 26],
 [27, 383, 1, 120, 15767, 3725, 1, 139, 2803, 7573],
 [769, 2877, 3965, 5862, 3772, 13, 867, 15768],
 [46, 263, 228, 635, 1, 1078, 274, 2818, 34, 93, 1359, 13, 594],
 [128, 3, 149, 97, 113, 66, 1725, 57, 13296],
 [63, 19, 30, 1948, 9, 3773, 2035, 344]]

In [48]:
vocab_lengthTitle=len(tok.index_word.keys())
vocab_lengthTitle

82593

In [49]:
sequences_matrix_train_Title = sequence.pad_sequences(sequences_train_Title,maxlen=maxLenTitle)
sequences_matrix_train_Title

array([[    0,     0,     0, ...,    39, 13295, 35156],
       [    0,     0,     0, ...,   435,   326,    23],
       [    0,     0,     0, ...,   620,    14,  3998],
       ...,
       [    0,     0,     0, ...,    10,    74,  2835],
       [    0,     0,     0, ...,     2,    18,    81],
       [    0,     0,     0, ..., 14559,   794,   245]], dtype=int32)

In [50]:
sequences_matrix_train_Title[:10]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     4,     1,    24, 13295, 35155,    39,
        13295, 35156],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,    35,    15,   165,     1,  3350,    10,   435,
          326,    23],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     4,     1,   690,    19,   332,   620,
           14,  3998],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,    46,   134,    15,    98,    67,    31,
            2,    21],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,   438,     1,   424,     7,
          579,    26],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,    27,   383,     1,   120, 15767,  3725,     1,   139,
         2803,  7573],
       [  

In [51]:
sequences_test_Title = tok.texts_to_sequences(X_test_Title)
sequences_matrix_test_Title = sequence.pad_sequences(sequences_test_Title,maxlen=maxLenTitle)

In [52]:
sequences_matrix_train_Title.shape,sequences_matrix_test_Title.shape,y_train.shape,y_test.shape

((529752, 20), (176584, 20), (529752, 10), (176584, 10))

In [53]:
y_train

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [54]:
sentLensTitleBody=[]
for line in train['Body']:
    sentLensTitleBody.append(len(word_tokenize(line)))
max(sentLensTitleBody)

14682

In [55]:
maxLenBodyTemp = int(np.quantile(sentLensTitleBody,0.90))

In [56]:
maxLenBodyTemp

602

In [57]:
maxLenBody = maxLenBodyTemp + (10-(maxLenBodyTemp % 10))
maxLenBody

610

In [58]:
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train_Body)
sequences_train_Body = tok.texts_to_sequences(X_train_Body)

In [59]:
vocab_lengthBody =len(tok.index_word.keys())
vocab_lengthBody

1655486

In [60]:
sequences_matrix_train_Body = sequence.pad_sequences(sequences_train_Body,maxlen=maxLenBody)
sequences_matrix_train_Body

array([[     0,      0,      0, ...,    394,   2876, 881084],
       [     0,      0,      0, ...,      2,     56, 126139],
       [     0,      0,      0, ...,      9,   3514,   3577],
       ...,
       [     0,      0,      0, ...,     24,     72,    738],
       [     0,      0,      0, ...,    202,     62,     10],
       [     0,      0,      0, ...,    182,    203,     10]], dtype=int32)

In [61]:
sequences_test_Body = tok.texts_to_sequences(X_test_Body)
sequences_matrix_test_Body = sequence.pad_sequences(sequences_test_Body,maxlen=maxLenBody)

In [62]:
sequences_matrix_train_Title.shape,sequences_matrix_train_Body.shape,y_train.shape

((529752, 20), (529752, 610), (529752, 10))

In [63]:
sequences_matrix_test_Title.shape,sequences_matrix_test_Body.shape,y_test.shape

((176584, 20), (176584, 610), (176584, 10))

In [64]:
from tensorflow.keras import initializers

In [65]:
maxLenTitle

20

In [66]:
def RNN():
    # Title Only
    title_input = Input(name='title_input',shape=[maxLenTitle])
    # title_Embed = Embedding(vocab_lengthTitle+1,2000,input_length=maxLenTitle,mask_zero=True,name='title_Embed')(title_input)
    title_Embed = Embedding(vocab_lengthTitle+1,2000,input_shape=(maxLenTitle,),mask_zero=True,name='title_Embed')(title_input)
    gru_out_t = GRU(300)(title_Embed)
    # auxiliary output to tune GRU weights smoothly
    #auxiliary_output = Dense(len(labels), activation='softmax', name='aux_output')(gru_out_t)
    auxiliary_output = Dense(len(labels), activation='sigmoid', name='aux_output')(gru_out_t)

    # Body Only
    body_input = Input(name='body_input',shape=[maxLenBody])
    #body_Embed = Embedding(vocab_lengthBody+1,170,input_length=maxLenBody,mask_zero=True,name='body_Embed')(body_input)
    body_Embed = Embedding(vocab_lengthBody+1,170,input_shape=(maxLenTitle,),mask_zero=True,name='body_Embed')(body_input)
    
    gru_out_b = GRU(200)(body_Embed)

    # combined with GRU output
    com = concatenate([gru_out_t, gru_out_b])

    # now the combined data is being fed to dense layers
    #dense1 = Dense(600,activation='relu',kernel_initializer=initializers.GlorotUniform(seed=20))(com)
    dense1 = Dense(400,activation='relu')(com)
    #dp1 = Dropout(0.3)(dense1)
    dp1 = Dropout(0.5)(dense1)
    bn = BatchNormalization()(dp1)
    dense2 = Dense(150,activation='relu')(bn)

    main_output = Dense(len(labels), activation='sigmoid', name='main_output')(dense2)
    #main_output = Dense(len(labels),activation='softmax', name='main_output')(dense2)

    model = Model(inputs=[title_input, body_input],outputs=[main_output, auxiliary_output])
    return model

In [67]:
model = RNN()
model.summary()

/Users/jaanavnalina/Learning/IITK/DeepLearning/01_StackOverFlow/venv/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ title_input         │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ body_input          │ (None, 610)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ title_Embed         │ (None, 20, 2000)  │ 165,188,0… │ title_input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 20)        │          0 │ title_input[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ body_Embed          │ (None, 610, 170)  │ 281,432,7… │ body_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 610)       │          0 │ body_input[0][0]  │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru (GRU)           │ (None, 300)       │  2,071,800 │ title_Embed[0][0… │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_1 (GRU)         │ (None, 200)       │    223,200 │ body_Embed[0][0], │
│                     │                   │            │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 500)       │          0 │ gru[0][0],        │
│ (Concatenate)       │                   │            │ gru_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 400)       │    200,400 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 400)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 400)       │      1,600 │ dropout[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 150)       │     60,150 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ main_output (Dense) │ (None, 10)        │      1,510 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ aux_output (Dense)  │ (None, 10)        │      3,010 │ gru[0][0]         │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 449,182,460 (1.67 GB)

 Trainable params: 449,181,660 (1.67 GB)

 Non-trainable params: 800 (3.12 KB)

In [68]:
model.compile(optimizer='adam',loss={'main_output': 'categorical_crossentropy', 'aux_output': 'categorical_crossentropy'},
              metrics=['accuracy','accuracy'])

In [69]:
results=model.fit({'title_input': sequences_matrix_train_Title, 'body_input': sequences_matrix_train_Body},
          {'main_output': y_train, 'aux_output': y_train},
          validation_data=[{'title_input': sequences_matrix_test_Title, 'body_input': sequences_matrix_test_Body},
          {'main_output': y_test, 'aux_output': y_test}],
          epochs=5, batch_size=800)

Epoch 1/5


/Users/jaanavnalina/Learning/IITK/DeepLearning/01_StackOverFlow/venv/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 165188000 elements. This may consume a large amount of memory.
  warnings.warn(
/Users/jaanavnalina/Learning/IITK/DeepLearning/01_StackOverFlow/venv/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 281432790 elements. This may consume a large amount of memory.
  warnings.warn(


273/663 ━━━━━━━━━━━━━━━━━━━━ 3:15:45 30s/step - aux_output_accuracy: 0.4607 - loss: 7.0335 - main_output_accuracy: 0.3809

KeyboardInterrupt: 

In [ ]:
(predicted_main, predicted_aux)=model.predict({'title_input': sequences_matrix_test_Title, 'body_input': sequences_matrix_test_Body},verbose=1)



In [ ]:
from sklearn.metrics import classification_report,f1_score

In [ ]:
print(f1_score(y_test,predicted_main>.55,average='samples'))

In [ ]:
print(classification_report(y_test,predicted_main>.55))

In [ ]:
test.iloc[30]

In [ ]:
predicted_main[30].round(decimals = 2)

In [ ]:
labels

In [ ]:
# model.save('/content/drive/My Drive/Colab Notebooks/Projects/01 Multiclass Multilabel prediction For stack overflow Questions/stackoverflow_tags.h5')
model.save('stackoverflow_tags.h5')

In [ ]:
eTime = time.time()
time_taken = eTime-sTime
print(f'Total time consumed: {time_taken:.0f} minute(s)')